In [ ]:
import numpy as np
import pandas as pd
import torch
import re
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/WELFake_Dataset.csv")
print(df.columns)
print(df.shape)
print(type(df.loc[7,"text"]))

text_iterator = df['text']
title_iterator = df['title']

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')
(72134, 4)
<class 'str'>


In [ ]:
# with stemming and removing stop words with nltk
ps = PorterStemmer()
stop_words = stopwords.words('english')

def process_text(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # lowercase
            t = t.lower()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # stem the words
            new_t = []
            for w in t:
              new_word = ps.stem(w)
              if new_word not in (stop_words):
                new_t.append(new_word)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text(text_iterator)
title_processed = process_text(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

72134
72134
demonstr gather last night exercis constitut protect right peac protest order rais issu creat chang loretta lynch aka eric holder skirt


In [ ]:
# just stemming of words
def process_text2(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9_.,!\"\'/$]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # stem the words
            new_t = []
            for w in t:
              new_word = ps.stem(w)
              new_t.append(new_word)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text2(text_iterator)
title_processed = process_text2(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

In [ ]:
# no stemming and removing stop words
def process_text3(text_iterator):
    text_processed = []
    for text in text_iterator:
        try:
            # remove punctuation
            t = re.sub("[^a-zA-Z0-9_.,!\"\'/$]", " ", text)
            # remove multiple spaces
            t = re.sub(r" +", " ", t)
            # remove newline
            t = re.sub(r"\n", " ", t)
            # clear trailing whitespaces
            t = t.strip()
            # tokenise
            t = t.split(" ")
            # drop empty string
            t = list(filter(lambda x: x != "", t))
            # join back the string
            new_t = []
            for w in t:
              new_t.append(w)
            # rejoin tokenized string
            t = " ".join(new_t)
            if len(t) == 0:
                text_processed.append("")
            else:
                text_processed.append(t)
        except:
            text_processed.append("")
            continue
    return text_processed


text_processed = process_text3(text_iterator)
title_processed = process_text3(title_iterator)

print(len(text_processed))
print(len(title_processed))
print(text_processed[2])

72134
72134
Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change. Loretta Lynch aka Eric Holder in a skirt


In [ ]:
merged_title_text = []
for i,v in enumerate(title_processed):
    if (v == None):
        merge = text_processed[i]
    elif (text_processed[i]==None):
        merge = v
    else:
        merge = v + " " + text_processed[i]
    merged_title_text.append(merge)

print(merged_title_text[2])

UNBELIEVABLE! OBAMA S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE PEACEFUL PROTESTERS In Her Home State Of North Carolina VIDEO Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change. Loretta Lynch aka Eric Holder in a skirt


In [ ]:
df_output = pd.DataFrame(columns=["title", "text", "merged", "class"])
df_output["title"] = title_processed
df_output["text"] = text_processed
df_output["merged"] = merged_title_text
df_output["class"] = df["label"]

In [ ]:
# split test and training dataset
# separator = 10000
separator = round(0.8*len(df_output["merged"]))

x_train = np.array(df_output["merged"][:separator])
y_train = []
ls = []
for i in df_output["class"][:separator]:
  ls.append(i)
  y_train.append(ls)
  ls = []
y_train = np.array(y_train)

x_test = np.array(df_output["merged"][separator+1:])
y_test = []
ls = []
for i in df_output["class"][separator+1:]:
  ls.append(i)
  y_test.append(ls)
  ls = []
y_test = np.array(y_test)

print(x_train[0:3])
print(y_test[0:3])

['LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9 11By BlackLivesMatter And FYF911 Terrorists VIDEO No comment is expected from Barack Obama Members of the FYF911 or FukYoFlag and BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to turn the tide and kill white people and cops to send a message about the killing of black people in America.One of the F YoFlag organizers is called Sunshine. She has a radio blog show hosted from Texas called, Sunshine s F ing Opinion Radio Show. A snapshot of her FYF911 LOLatWhiteFear Twitter page at 9 53 p.m. shows that she was urging supporters to Call now!! fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite FYF911The radio show aired at 10 00 p.m. eastern standard time.During the show, callers clearly call for lynching and killing of white people.A 2 39 minute clip from the rad

In [ ]:
# get text length
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

# print((get_text_length(x_test[5:15])))


In [ ]:
# get pos of words
def get_pos_words(x):
  n_ctr, a_ctr, v_ctr, r_ctr, others = 0, 0, 0, 0, 0
  classified = ""
  counters = []
  output = []
  for t in x:
    counters = []
    n_ctr, a_ctr, v_ctr, r_ctr, others = 0, 0, 0, 0, 0
    for w in t.split(" "):
      if wn.synsets(w):
        classified = wn.synsets(w)[0].pos()
        if classified == "n": n_ctr+=1
        elif classified == "a": a_ctr+=1
        elif classified == "v": v_ctr+=1
        elif classified == "r": r_ctr+=1
      else: others+=1
    counters.append(n_ctr/len(t.split(" ")))
    counters.append(a_ctr/len(t.split(" ")))
    counters.append(v_ctr/len(t.split(" ")))
    counters.append(r_ctr/len(t.split(" ")))
    counters.append(others/len(t.split(" ")))
    output.append(counters)
  return np.array(output)
  # return np.array([wn.synsets(w)[0].pos() for w in x]).reshape(-1, -1)
# for w in words:
#     tmp = wn.synsets(w)[0].pos()
#     print w, ":", tmp
# print(get_pos_words(x_test[5:15]))

In [ ]:
# get no. of capital letters
def get_capital_letters(x):
  counter = 0
  total = []
  for t in x:
    number = []
    counter = 0
    for l in t:
      if l.isupper(): counter+=1
    number.append(counter/len(t))
    total.append(number)
  return np.array(total)

# print((get_capital_letters(x_test[5:10])))


In [ ]:
# normal bag of words model
classifier = Pipeline([
    ('vectorizer', CountVectorizer(min_df=0,max_df=1.0)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())])
classifier.fit(x_train, y_train.ravel())

predicted = classifier.predict(x_test)
num_correct = 0
for i in range(len(predicted)):
  if predicted[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12481
0.8651739914044088


In [ ]:
# bag of words model with text length
classifier2 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=1.0)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier2.fit(x_train, y_train.ravel())
predicted2 = classifier2.predict(x_test)
# predicted

num_correct = 0
for i in range(len(predicted2)):
  if predicted2[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12350
0.856093165118536


In [ ]:
# normal bag of words model with length and pos
classifier3 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=1.0)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier3.fit(x_train, y_train.ravel())

predicted3 = classifier3.predict(x_test)
num_correct = 0
for i in range(len(predicted3)):
  if predicted3[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12335
0.8550533758491612


In [ ]:
# normal bag of words model with length and pos and proportion of capital letters
classifier4 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=1.0)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier4.fit(x_train, y_train.ravel())

predicted4 = classifier4.predict(x_test)
num_correct = 0
for i in range(len(predicted4)):
  if predicted4[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12337
0.8551920144184112


In [ ]:
# normal bag of words model and proportion of capital letters
classifier5 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=1.0)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier5.fit(x_train, y_train.ravel())

predicted5 = classifier5.predict(x_test)
num_correct = 0
for i in range(len(predicted5)):
  if predicted5[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12506
0.8669069735200333


In [ ]:
# normal bag of words model and proportion of capital letters
classifier6 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=0,max_df=0.95)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('capital', Pipeline([
            ('count', FunctionTransformer(get_capital_letters, validate=False)),
        ])),
        ('pos', Pipeline([
            ('count', FunctionTransformer(get_pos_words, validate=False)),
        ]))
    ])),
    ('clf', MultinomialNB())])

classifier6.fit(x_train, y_train.ravel())

predicted6 = classifier6.predict(x_test)
num_correct = 0
for i in range(len(predicted6)):
  if predicted6[i] == y_test[i]:
    num_correct += 1

print(num_correct)
print(num_correct/len(y_test))

12510
0.8671842506585332
